<a href="https://colab.research.google.com/github/Manu-Anand/SarcomaGithub/blob/master/10b_tranformers_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://noisy-text.github.io/2017/files/wnut17train.conll
# https://huggingface.co/transformers/custom_datasets.html

--2021-04-15 12:30:37--  http://noisy-text.github.io/2017/files/wnut17train.conll
Resolving noisy-text.github.io (noisy-text.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to noisy-text.github.io (noisy-text.github.io)|185.199.108.153|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 493781 (482K) [application/octet-stream]
Saving to: ‘wnut17train.conll’

wnut17train.conll   100%[===================>] 482.21K  --.-KB/s    in 0.008s  

2021-04-15 12:30:38 (56.1 MB/s) - ‘wnut17train.conll’ saved [493781/493781]



In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 15.7MB/s 
     |████████████████████████████████| 870kB 50.8MB/s 
     |████████████████████████████████| 3.3MB 36.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=8c64f98987780484bab3cdeaa55c1bfc9bfb8d7528b825de6affbd249a16e84d
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
import re
from pathlib import Path

def read_wnut(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        #print(doc)
        #break
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token, tag = line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

texts, tags = read_wnut('wnut17train.conll')

In [ ]:
print(texts[0][10:17], tags[0][10:17], sep='\n')
print(texts[0])

['for', 'two', 'weeks', '.', 'Empire', 'State', 'Building']
['O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location']
['@paulwalk', 'It', "'s", 'the', 'view', 'from', 'where', 'I', "'m", 'living', 'for', 'two', 'weeks', '.', 'Empire', 'State', 'Building', '=', 'ESB', '.', 'Pretty', 'bad', 'storm', 'here', 'last', 'evening', '.']


In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_tags, val_tags = train_test_split(texts, tags, test_size=.2)

create encodings for our tokens and tags. For the tags, we can start by just create a simple mapping which we’ll use in a moment:

In [ ]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}
print(id2tag)

{0: 'I-group', 1: 'B-group', 2: 'I-creative-work', 3: 'I-person', 4: 'B-product', 5: 'I-product', 6: 'B-corporation', 7: 'B-person', 8: 'B-creative-work', 9: 'O', 10: 'I-corporation', 11: 'I-location', 12: 'B-location'}


In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)